In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(777)  # reproducibility

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
class Model:
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        learning_rate = 0.001
        with tf.variable_scope(self.name):
            self.X = tf.placeholder(tf.float32,shape=(None,784))
            self.X_img = tf.reshape(self.X,shape=(-1,28,28,1))
            self.Y = tf.placeholder(tf.float32,shape=(None,10))
            self.training = tf.placeholder(tf.bool)
            
            # conv layer1
            conv1 = tf.layers.conv2d(inputs=self.X_img,filters=32,kernel_size=[3,3],padding='SAME',activation=tf.nn.relu,strides=1)
            pool1 = tf.layers.max_pooling2d(inputs=conv1,pool_size=[2,2],padding='SAME',strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,rate=0.7,training=self.training)
            
            # conv layer2
            conv2 = tf.layers.conv2d(inputs=dropout1,filters=64,kernel_size=[3,3],padding='SAME',activation=tf.nn.relu,strides=1)
            pool2 = tf.layers.max_pooling2d(inputs=conv2,pool_size=[2,2],padding='SAME',strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,rate=0.7,training=self.training)
            
            # conv layer3
            conv3 = tf.layers.conv2d(inputs=dropout2,filters=128,kernel_size=[3,3],padding='SAME',activation=tf.nn.relu,strides=1)
            pool3 = tf.layers.max_pooling2d(inputs=conv3,pool_size=[2,2],padding='SAME',strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,rate=0.7,training=self.training)
            
            # DNN
            flat = tf.reshape(dropout3,shape=(-1,128*4*4))
            dense4 = tf.layers.dense(inputs=flat,units=625,activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,rate=0.5,training=self.training)
            
            # Fully Connected Network
            W = tf.Variable(tf.random_normal([625,10]))
            b = tf.Variable(tf.random_normal([10]))
            self.logits = tf.matmul(dropout4,W)+b
            
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits,labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            
            prediction = tf.argmax(self.logits,1)
            is_correct = tf.equal(prediction,tf.argmax(self.Y,1))
            self.accuracy = tf.reduce_mean(tf.cast(is_correct,dtype=tf.float32))
    
    
    def predict(self,x_test,training=False):
        return self.sess(self.logits,feed_dict={
            self.X : x_test,
            self.training : training
        })
    
    def get_accuracy(self,x_test,y_test,training=False):
        return self.sess(self.accuracy,feed_dict={
            self.X : x_test,
            self.Y : y_test,
            self.training : training
        })

    def train(self,x_data,y_data,training=True):
        return sess.run([self.cost,self.optimizer],feed_dict={
            self.X : x_data,
            self.Y : y_data,
            self.training : training
        })


In [3]:
#hyperparameter
training_epoches = 15
batch_size = 100

#make models
models =[]
num_models = 2
sess = tf.Session()
for i in range(num_models):
    models.append(Model(sess,'Model' + str(i)))
    
sess.run(tf.global_variables_initializer())

print('Start learning')
for epoch in range(training_epoches):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples / batch_size)
    for i in range(total_batch):
        batch_xs,batch_ys = mnist.train.next_batch(batch_size)
        for m_idx,m in enumerate(models):
            cost_val, _ = m.train(batch_xs,batch_ys)
            avg_cost_list[m_idx] += (cost_val / total_batch)
    print('Epoch : ',epoch+1,'cost : ',avg_cost_list)
                  
print('Learning Finished!')

    
test_size = len(mnist.test.labels)
predictions = np.zeros(test_size*10).reshape((test_size,10))
for idx,m in enumerate(models):
    print(m_idx, 'Accuracy:', m.get_accuracy(
        mnist.test.images, mnist.test.labels))
    p = m.predict(mnist.test.images) # return logits as list
    predictions += p

prediction = tf.argmax(predictions,1)
is_correct = tf.equal(prediction,tf.argmax(mnist.test.labels,1))
accuracy = tf.reduce_mean(tf.cast(is_correct,dtype=tf.float32))
print('Ensemble accuracy:', sess.run(ensemble_accuracy))

Start learning
Epoch :  1 cost :  [ 1.67804132  1.30921945]
Epoch :  2 cost :  [ 0.54401284  0.45103777]


KeyboardInterrupt: 